<a href="https://colab.research.google.com/github/drfrostongithub/collab/blob/main/AHP_Untuk_Penilaian_Risiko_Proyek_%2B_SWOT_berbobot_%2B_Tolerance_for_Float_Point_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def input_criteria():
    print("=== Masukkan Kriteria Risiko ===")
    criteria = []
    n = int(input("Berapa banyak kriteria risiko yang ingin Anda evaluasi? "))
    for i in range(n):
        nama = input(f"Nama kriteria ke-{i+1}: ")
        criteria.append(nama)
    return criteria

def buat_matriks_perbandingan(criteria):
    n = len(criteria)
    matrix = np.ones((n, n))

    print("\n=== Input Perbandingan Berpasangan (gunakan skala Saaty 1-9 atau 1/n) ===")
    for i in range(n):
        for j in range(i+1, n):
            while True:
                try:
                    nilai = float(input(f"Seberapa penting '{criteria[i]}' dibanding '{criteria[j]}' (1–9 atau 1/n)? "))
                    matrix[i][j] = nilai
                    matrix[j][i] = 1 / nilai
                    break
                except:
                    print("Input tidak valid. Masukkan angka.")

    return matrix

def hitung_prioritas(matrix):
    # Untuk Normalisasi
    kolom_jumlah = np.sum(matrix, axis=0)
    matrix_norm = matrix / kolom_jumlah
    bobot = np.mean(matrix_norm, axis=1)
    return bobot, matrix_norm

def hitung_konsistensi(matrix, bobot):
    n = len(bobot)
    hasil = np.dot(matrix, bobot)
    lambda_max = np.sum(hasil / bobot) / n
    CI = (lambda_max - n) / (n - 1)
    RI_table = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24,
                7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    RI = RI_table.get(n, 1.49)  # default max RI dari Standar SAATY
    CR = CI / RI if RI != 0 else 0
    return lambda_max, CI, CR

def tampilkan_hasil(criteria, bobot, lambda_max, CI, CR):
    print("\n=== Hasil AHP ===")
    print("Bobot Prioritas:")
    for i, b in enumerate(bobot):
        print(f"- {criteria[i]}: {round(b, 4)}")

    print(f"\nNilai λ_max = {round(lambda_max, 4)}")
    print(f"Consistency Index (CI) = {round(CI, 4)}")
    print(f"Consistency Ratio (CR) = {round(CR, 4)}")

    if CR < 0.1:
        print("Keputusan Anda konsisten.")
    else:
        print("Keputusan Anda *tidak konsisten !!!. Anda perlu evaluasi ulang input.")

def main():
    while True:
        criteria = input_criteria()
        matrix = buat_matriks_perbandingan(criteria)
        bobot, matrix_norm = hitung_prioritas(matrix)
        lambda_max, CI, CR = hitung_konsistensi(matrix, bobot)
        tampilkan_hasil(criteria, bobot, lambda_max, CI, CR)

        lanjut = input("\n Apakah Anda ingin menghitung ulang dengan data baru? (y/n): ").lower()
        if lanjut != 'y':
            print("Terima kasih telah menggunakan metode AHP !")
            break

# Program Main
main()


=== Masukkan Kriteria Risiko ===

=== Input Perbandingan Berpasangan (gunakan skala Saaty 1-9 atau 1/n) ===
Input tidak valid. Masukkan angka.
Input tidak valid. Masukkan angka.
Input tidak valid. Masukkan angka.


In [ ]:
def input_faktor_swot(kategori, sisa_bobot):
    print(f"\n Masukkan faktor SWOT untuk kategori: {kategori}")
    faktor_list = []
    tolerance = 1e-9

    while abs(sisa_bobot) > tolerance:
        faktor = input(f"- {kategori} (kosongkan untuk selesai): ").strip()
        if faktor == "":
            break

        print(f"  Sisa bobot yang tersedia: {sisa_bobot:.2f}")
        try:
            bobot = float(input("  Masukkan bobot (0.0 - 1.0): "))
            if bobot <= 0 or bobot > sisa_bobot + tolerance:
                print (sisa_bobot-bobot)
                print(f"  Bobot tidak valid. Maksimum yang bisa dimasukkan: {sisa_bobot:.2f}")
                continue
            skor = int(input("  Masukkan skor (1–4): "))
            if skor < 1 or skor > 4:
                print("  Skor harus antara 1 hingga 4.")
                continue

            faktor_list.append({
                'kategori': kategori,
                'faktor': faktor,
                'bobot': bobot,
                'skor': skor,
                'nilai': bobot * skor
            })

            sisa_bobot -= bobot
        except:
            print("  Input tidak valid. Ulangi.")

    if sisa_bobot <= tolerance:
        print(" Total bobot sudah mencapai 1.00, tidak bisa menambah faktor lagi.")

    return faktor_list, sisa_bobot

def cetak_hasil(faktor_all):
    print("\n Hasil Weighted SWOT Analysis")
    print(f"{'Kategori':<10} {'Faktor':<40} {'Bobot':<6} {'Skor':<4} {'Bobot×Skor':<10}")
    print("-"*75)
    total = 0
    for f in faktor_all:
        print(f"{f['kategori']:<10} {f['faktor']:<40} {f['bobot']:<6.2f} {f['skor']:<4} {f['nilai']:<10.2f}")
        total += f['nilai']
    print("-"*75)
    print(f"{'TOTAL SKOR SWOT':<62} = {total:.2f}")

    # Rekomendasi strategi
    if total >= 3.0:
        saran = "Strategi ofensif/agresif (growth)"
    elif 2.0 <= total < 3.0:
        saran = "Strategi bertahan atau pengembangan selektif"
    else:
        saran = "Strategi defensif atau efisiensi"

    print(f"\n Rekomendasi strategi: {saran}")

def main():
    print("=== Weighted SWOT Analysis ===")
    semua_faktor = []
    sisa_bobot = 1.0

    for kategori in ['Strength', 'Weakness', 'Opportunity', 'Threat']:
        if sisa_bobot <= 0:
            print(f"\n Tidak bisa menambahkan faktor {kategori} karena total bobot sudah mencapai 1.00.")
            break
        faktor_kat, sisa_bobot = input_faktor_swot(kategori, sisa_bobot)
        semua_faktor.extend(faktor_kat)

    cetak_hasil(semua_faktor)

    # Tanya lanjut
    ulang = input("\n Mau analisis SWOT lain? (y/n): ").lower()
    if ulang == 'y':
        print("\n Mengulang analisis.\n")
        main()
    else:
        print("Analisis selesai. Terima kasih!")

# Jalankan Main function
main()


=== Weighted SWOT Analysis ===

 Masukkan faktor SWOT untuk kategori: Strength
- Strength (kosongkan untuk selesai): Reputasi RS kuat
  Sisa bobot yang tersedia: 1.00
  Masukkan bobot (0.0 - 1.0): 0.15
  Masukkan skor (1–4): 4
- Strength (kosongkan untuk selesai): Tim pengembang berpengalaman
  Sisa bobot yang tersedia: 0.85
  Masukkan bobot (0.0 - 1.0): 0.10
  Masukkan skor (1–4): 3
- Strength (kosongkan untuk selesai): 

 Masukkan faktor SWOT untuk kategori: Weakness
- Weakness (kosongkan untuk selesai): Ketergantungan pada API eksternal
  Sisa bobot yang tersedia: 0.75
  Masukkan bobot (0.0 - 1.0): 0.15
  Masukkan skor (1–4): 2
- Weakness (kosongkan untuk selesai): SDM internal kurang siap
  Sisa bobot yang tersedia: 0.60
  Masukkan bobot (0.0 - 1.0): 0.10
  Masukkan skor (1–4): 1
- Weakness (kosongkan untuk selesai): 

 Masukkan faktor SWOT untuk kategori: Opportunity
- Opportunity (kosongkan untuk selesai): Tren digitalisasi kesehatan
  Sisa bobot yang tersedia: 0.50
  Masukkan bo